# Wholesale HP Past 60

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import statsmodels.api as sm
import seaborn as sns
import datetime

In [2]:
bid=pd.read_csv('bid_ytd.csv')
stock=pd.read_csv('stock_ytd.csv')

bid_mj=pd.read_csv('2021mayjunebids.csv')
stock_mj=pd.read_csv('2021mayjunequotes.csv')

In [3]:
stock['date']=pd.to_datetime(stock['date'])

# bid to datetime
bid['date']=pd.to_datetime(bid['bid_placed_on']).dt.date
bid['bid_placed_on']=pd.to_datetime(bid['bid_placed_on'])
bid_mj['bid_placed_on']=pd.to_datetime(bid_mj['bid_placed_on'])
bid_mj['date']=pd.to_datetime(bid_mj['bid_placed_on']).dt.date

# lowercase
bid_mj['buyer']=bid_mj['buyer'].str.lower()

In [4]:
# bid_all_ytd=bid.merge(bid_mj,how='outer')

# bid_all_ytd.to_csv('bid_ytd.csv')

In [5]:
# stock_all_ytd=stock.merge(stock_mj, how='outer')
# stock_all_ytd.to_csv('stock_ytd.csv')

In [6]:
start=datetime.datetime(2021,4,1)

stock_apr=stock[stock['date']>=start]

bid_apr=bid[bid['bid_placed_on']>=start].drop(columns=['day','hour','day']).reset_index(drop=True)

In [7]:
stock_60=stock_apr.merge(stock_mj, how='outer')
stock_60.loc[stock_60['brand_name']=='Saint Laurent', 'brand_name']='Yves Saint Laurent'

bid_60=bid_apr.merge(bid_mj, how='outer')
bid_60.loc[bid_60['brand_name']=='Saint Laurent', 'brand_name']='Yves Saint Laurent'

In [8]:
stock_category=(pd.DataFrame(stock_60.groupby('category_name')['price']
                       .sum()
                       .rename('stock_gmv')
                       .sort_values(ascending=False)))

stock_category['items_sold']=stock_60.groupby('category_name')['price'].count()

In [9]:
all_win=bid_60.loc[bid_60['bid']>=bid_60['winning_price']]

auc_category=(pd.DataFrame(all_win.groupby('category_name')['winning_price']
                       .sum()
                       .rename('auction_gmv')
                       .sort_values(ascending=False)))

auc_category['items_won']=all_win.groupby('category_name')['winning_price'].count()

auc_category.head()

,auction_gmv,items_won
category_name,,
Shoulder Bags,684967.0,705
Totes,387426.0,474
Travel,232044.0,320
Handbags,168472.0,174
Top Handles & Satchels,130508.0,180


In [10]:
category=auc_category.merge(stock_category, left_index=True, right_index=True, how='outer')

category=category.fillna(0)
category['total_gmv']=category['stock_gmv']+category['auction_gmv']
category['total_items_sold']=category['items_sold']+category['items_won']

category=category.sort_values(by='total_gmv', ascending=False)
cols=['auction_gmv', 'stock_gmv', 'total_gmv','items_sold','items_won', 'total_items_sold']
category=category[cols]
category.head()

,auction_gmv,stock_gmv,total_gmv,items_sold,items_won,total_items_sold
category_name,,,,,,
Shoulder Bags,684967.0,1391184.40,2076151.40,1191.0,705.0,1896.0
Totes,387426.0,299964.75,687390.75,383.0,474.0,857.0
Top Handles & Satchels,130508.0,299350.00,429858.00,176.0,180.0,356.0
Handbags,168472.0,185701.48,354173.48,274.0,174.0,448.0
Travel,232044.0,86724.70,318768.70,158.0,320.0,478.0


In [17]:
# category.to_excel('60_stockauc_categories.xlsx')

In [12]:
stock_brand=(pd.DataFrame(stock_60.groupby('brand_name')['price']
                       .sum()
                       .rename('stock_gmv')
                       .sort_values(ascending=False)))

stock_brand['items_sold']=stock_60.groupby('brand_name')['price'].count()

stock_brand.head()

,stock_gmv,items_sold
brand_name,,
Chanel,827070.11,501
Louis Vuitton,792398.29,1434
Hermès,607141.77,450
Gucci,229058.98,520
Christian Dior,66775.34,71


In [13]:
bid_brand=(pd.DataFrame(all_win.groupby('brand_name')['winning_price']
                       .sum()
                       .rename('auction_gmv')
                       .sort_values(ascending=False)))

bid_brand['items_won']=all_win.groupby('brand_name')['winning_price'].count()

bid_brand

,auction_gmv,items_won
brand_name,,
Louis Vuitton,1118284.0,1669
Chanel,488730.0,646
Hermès,299343.0,546
Gucci,143156.0,303
Prada,33471.0,66
Goyard,26005.0,28
Christian Dior,24665.0,46
Fendi,23727.0,69
Van Cleef & Arpels,16435.0,9


In [14]:
brand=stock_brand.merge(bid_brand, left_index=True, right_index=True, how='outer')

brand=brand.fillna(0)
brand['total_gmv']=brand['stock_gmv']+brand['auction_gmv']
brand['total_items_sold']=brand['items_sold']+brand['items_won']

brand=brand.sort_values(by='total_gmv', ascending=False)
cols=['auction_gmv', 'stock_gmv', 'total_gmv','items_sold','items_won', 'total_items_sold']
brand=brand[cols]
brand.head()

,auction_gmv,stock_gmv,total_gmv,items_sold,items_won,total_items_sold
brand_name,,,,,,
Louis Vuitton,1118284.0,792398.29,1910682.29,1434.0,1669.0,3103.0
Chanel,488730.0,827070.11,1315800.11,501.0,646.0,1147.0
Hermès,299343.0,607141.77,906484.77,450.0,546.0,996.0
Gucci,143156.0,229058.98,372214.98,520.0,303.0,823.0
Prada,33471.0,66369.01,99840.01,166.0,66.0,232.0


In [16]:
# brand.to_excel('60_stockauc_brands.xlsx')